In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# шлях до тренувальних справжніх даних
real_path = ""
# шлях до тренувальних синтезованих даних
fake_path = ""
real = []
fake = []

# шлях до валідаційних справжніх даних
vreal_path = ""
# шлях до валідаційних синтезованих даних
vfake_path = ""
vreal = []
vfake = []

def get_audio(folder_path):
    raw = []
    for filename in os.listdir(folder_path):
        audio, sr = librosa.load(os.path.join(folder_path, filename), sr=None)
        audio_parts = np.array_split(audio, 4)
        for part in audio_parts:
            raw.append(part)
    return raw

real = get_audio(real_path)
fake = get_audio(fake_path)

vreal = get_audio(vreal_path)
vfake = get_audio(vfake_path)

In [ ]:
print("Кількість тренувальних аудіофайлів real:", len(real))
print("Кількість тренувальних аудіофайлів fake:", len(fake))

print("Кількість валідаційних аудіофайлів real:", len(vreal))
print("Кількість валідаційних аудіофайлів fake:", len(vfake))

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Побудова міток для класів real та fake
real_labels = np.zeros(len(real))
fake_labels = np.ones(len(fake))

vreal_labels = np.zeros(len(vreal))
vfake_labels = np.ones(len(vfake))

# Об'єднання даних та міток
X_train = np.concatenate((real, fake), axis=0)
y_train = np.concatenate((real_labels, fake_labels), axis=0)

X_val = np.concatenate((vreal, vfake), axis=0)
y_val = np.concatenate((vreal_labels, vfake_labels), axis=0)

# Перемішуємо тренувальні дані
shuffle_index_train = np.random.permutation(len(X_train))
X_train_shuffled = X_train[shuffle_index_train]
y_train_shuffled = y_train[shuffle_index_train]

# Перемішуємо валідаційні дані
shuffle_index_val = np.random.permutation(len(X_val))
X_val_shuffled = X_val[shuffle_index_val]
y_val_shuffled = y_val[shuffle_index_val]


# Виведення розмірності тренувальних та тестових даних

print("Розмірність тренувальних даних:", X_train_shuffled.shape)
print("Розмірність тестових даних:", X_val_shuffled.shape)
print("Розмірність міток тренувальних даних:", y_train_shuffled.shape)
print("Розмірність міток тестових даних:", y_val_shuffled.shape)

# X_train = X_train_shuffled[..., np.newaxis]
# X_test = X_val_shuffled[..., np.newaxis]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Побудова простої нейронної мережі
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_shuffled.shape[1],)),  # Повнозв'язний шар з 64 нейронами та функцією активації ReLU
    Dense(32, activation='relu'),  # Ще один повнозв'язний шар з 32 нейронами та функцією активації ReLU
    Dense(1, activation='sigmoid')  # Вихідний шар з одним нейроном для бінарної класифікації і функцією активації sigmoid
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train_shuffled, y_train_shuffled, epochs=10, batch_size=32, validation_data=(X_val_shuffled, y_val_shuffled))


In [ ]:
model.save("RawOverF-05.keras")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers

# Побудова складної нейронної мережі
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train_shuffled.shape[1],)),  # Повнозв'язний шар з 64 нейронами та функцією активації ReLU, з L2 регуляризацією
    Dropout(0.5),  # Dropout шар для уникнення перенавчання
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # Повнозв'язний шар з 32 нейронами та функцією активації ReLU, з L2 регуляризацією
    Dropout(0.5),  # Dropout шар для уникнення перенавчання
    Dense(1, activation='sigmoid')  # Вихідний шар з одним нейроном для бінарної класифікації і функцією активації sigmoid
])

# Компіляція моделі
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train_shuffled, y_train_shuffled, epochs=40, batch_size=32, validation_data=(X_val_shuffled, y_val_shuffled))


In [ ]:
model.save("Raw-05.keras")